#### API using Flask

In [2]:
import os
hello_world_script_file = os.path.join(os.path.pardir, 'src', 'modeling', 'hello_world_api.py')

In [2]:
%%writefile $hello_world_script_file

from flask import Flask, request

# Initialize the Flask app
app = Flask(__name__)

# Define an API endpoint that accepts POST requests
@app.route('/api', methods=['POST'])
def say_hello():
    # Parse incoming JSON data
    data = request.get_json(force=True)
    # Extract 'name' from the JSON and return a greeting
    name = data['name']
    return f'hello {name}'

# Run the app on port 10001 with debug mode enabled
if __name__ == '__main__':
    app.run(port=10001, debug=True)

Writing ..\src\modeling\hello_world_api.py


In [3]:
import json
import requests

In [6]:
url = 'http://127.0.0.1:10001/api'
data = json.dumps({'name': 'James'})
r = requests.post(url, data)

In [8]:
print(r.text)

hello James


#### Machine Learning API using Flask

In [10]:
machine_learning_script_file = os.path.join(os.path.pardir, 'src', 'modeling', 'machine_learning_api.py')

In [14]:
%%writefile $machine_learning_script_file

from flask import Flask, request
import pandas as pd
import os
import json
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pickle

# Initialize the Flask app
app = Flask(__name__)

# Define file paths
model_path = os.path.join(os.path.pardir, os.path.pardir, 'models')
model_fp = os.path.join(model_path, 'lr_model.pkl')
scaler_fp = os.path.join(model_path, 'lr_scaler.pkl')

scaler = pickle.load(open(scaler_fp, 'rb'))
model = pickle.load(open(model_fp, 'rb'))

# set column order to match pickle files
columns = [
    'Age', 'Fare', 'Family_Size', 'Is_Mother', 'Is_Male', 'Is_Female', 
    'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 
    'Deck_Z', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Lady', 'Title_Master', 
    'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer', 'Title_Sir', 
    'Fare_Bin_very_low', 'Fare_Bin_low', 'Fare_Bin_high', 'Fare_Bin_very_high', 
    'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Age_State_Adult', 
    'Age_State_Child', 'Age_State_Senior', 'Age_State_Teenager'
]


# Define an API endpoint that accepts POST requests
@app.route('/api', methods=['POST'])
def make_prediction():
    # Parse incoming JSON data
    data = json.dumps(request.get_json(force=True))
    # ceate df from json string
    df = pd.read_json(data)
    # extract passengerId's
    passenger_ids = df['PassengerId'].ravel()
    #actual survived values
    actuals = df['Survived'].ravel()
    # Extract required columns as numpy array
    X = df[columns].to_numpy().astype('float')
    # transform the input
    X_scaled = scaler.transform(X)
    # make predictions
    predictions = model.predict(X_scaled)
    # create response dataframe
    df_response = pd.DataFrame({
        'PassengerId': passenger_ids,
        'Predicted': predictions,
        'Actual': actuals
    })
    return df_response.to_json()

# Run the app on port 10001 with debug mode enabled
if __name__ == '__main__':
    app.run(port=10001, debug=True)

Overwriting ..\src\modeling\machine_learning_api.py


#### Invoking API using Requests

In [15]:
import pandas as pd
import os

processed_data_fp = os.path.join(os.path.pardir, 'data', 'processed')
train_fp = os.path.join(processed_data_fp, 'train.csv')
train_df = pd.read_csv(train_fp)

In [16]:
survived_passengers = train_df[train_df['Survived'] == 1][:5]

In [17]:
survived_passengers

,PassengerId,Survived,Age,Fare,Family_Size,Is_Mother,Is_Male,Is_Female,Deck_A,Deck_B,...,Fare_Bin_low,Fare_Bin_high,Fare_Bin_very_high,Embarked_C,Embarked_Q,Embarked_S,Age_State_Adult,Age_State_Child,Age_State_Senior,Age_State_Teenager
1,2,1.0,38.0,71.2833,2,0,0,1,0,0,...,0,0,1,1,0,0,1,0,0,0
2,3,1.0,26.0,7.9250,1,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
3,4,1.0,35.0,53.1000,2,0,0,1,0,0,...,0,0,1,0,0,1,1,0,0,0
8,9,1.0,27.0,11.1333,3,1,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
9,10,1.0,14.0,30.0708,2,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,1


In [18]:
import requests
def make_api_request(data):
    # url for api
    url = 'http://127.0.0.1:10001/api'
    # make post request
    r = requests.post(url, data)
    return r.json()

In [19]:
make_api_request(survived_passengers.to_json())

{'PassengerId': {'0': 2, '1': 3, '2': 4, '3': 9, '4': 10},
 'Predicted': {'0': 1.0, '1': 1.0, '2': 1.0, '3': 1.0, '4': 1.0},
 'Actual': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}}

In [24]:
# compare results
from io import StringIO
result = make_api_request(train_df.to_json())
df_result = pd.read_json(StringIO(json.dumps(result)))
df_result.head()

,PassengerId,Predicted,Actual
0,1,0,0
1,2,1,1
2,3,1,1
3,4,1,1
4,5,0,0


In [28]:
# accuracy level
import numpy as np
np.mean(df_result['Actual'] == df_result['Predicted'])

0.8428731762065096